In [1]:
import pandas as pd
from pathlib import Path
from typing import List
from natasha import MorphVocab
import pymorphy

In [2]:
path = Path('C:/may/ML/GroupLe/grouple/data/face_detection/raw/characters/you_anime_characters_refs.csv')

In [3]:
df = pd.read_csv(path, sep = '\t')

In [4]:
df['name'][:10]

0         Эрен Йегер
1    Микаса Аккерман
2       Армин Арлерт
3               Леви
4         Саша Блауз
5     Конни Спрингер
6             Ханнес
7     Бертольт Гувер
8         Эрвин Смит
9       Райнер Браун
Name: name, dtype: object

In [6]:
class Character:
    def __init__(self, name, pic, link):
        self.name = name
        self.pic = pic
        self.link = link

In [17]:
class CandidatesSearch:
    def __init__(self, df):
        self.df = df
        self.morph = pymorphy2.MorphAnalyzer()
        
    def lemmatize(self, word):
        p = self.morph.parse(word)[0]
        res = p.normal_form.capitalize()
        return res

    def get_info_name(self, name: str) -> List[Character]:
        df_full = self.df[self.df['name']==name]

        if df_full.empty:
            name_lemm = self.lemmatize(name)
            df_lemm = (df[df['name'].str.contains(name_lemm)])
            df_chars = df_lemm.drop_duplicates(subset = 'link')
        else: 
            df_chars = df_full.drop_duplicates(subset = 'link')

        characters = []
        for i, (page_link, name, link, img_addr) in df_chars.iterrows():
            char = Character(name, page_link[:20] + img_addr, page_link[:20] + link)
            characters.append(char)
        return characters

    def get_info(self, names: List[str]):
        info = []
        for name in names:
            info.append(self.get_info_name(name))
        return info

In [24]:
candidates_search = CandidatesSearch(df)
l = ['Эрвина', 'Леви', 'Мари', 'Бертольта']
a = candidates_search.get_info(l)
a[3][0].name

'Бертольт Гувер'